In [1]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.7 MB/s eta 0:00:00


In [2]:
!pip install -q supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 7.2 MB/s eta 0:00:00


In [3]:
!unzip /kaggle/input/seadronesseemot-dataset-video-conversion/videos.zip -d /kaggle/working/videos

Archive:  /kaggle/input/seadronesseemot-dataset-video-conversion/videos.zip
   creating: /kaggle/working/videos/kaggle/working/videos/
  inflating: /kaggle/working/videos/kaggle/working/videos/video_009.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_005.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_018.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_015.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_007.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_014.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_000.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_013.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_021.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_012.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_019.mp4  
  inflating: /kaggle/working/videos/kaggle/working/

In [4]:
import numpy as np
import cv2
from collections import deque
import supervision as sv
from ultralytics import YOLO
from pathlib import Path
import os

class NovelByteTrack(sv.ByteTrack):
    """
    COMPLETELY FIXED Novel ByteTrack - Proper OpenCV KalmanFilter
    """
    
    def __init__(self, track_thresh=0.5, track_buffer=30, match_thresh=0.8, frame_rate=30):
        super().__init__(track_thresh, track_buffer, match_thresh, frame_rate)
        self.kalman_filters = {}
        self.track_histories = {}
        self.age_threshold = 30
        self.speed_factor = 0.3
        
    def update_with_detections(self, detections):
        # 1. Pre-process detections
        detections = self._drone_preprocessing(detections)
        
        if len(detections) == 0:
            return detections
        
        # 2. Standard ByteTrack
        tracked_detections = super().update_with_detections(detections)
        
        # 3. Post-process
        tracked_detections = self._novel_postprocessing(tracked_detections)
        
        return tracked_detections
    
    def _drone_preprocessing(self, detections):
        """Sea surface reflection filtering"""
        if len(detections) == 0:
            return detections
        
        boxes = detections.xyxy
        confs = detections.confidence
        
        w = boxes[:, 2] - boxes[:, 0]
        h = boxes[:, 3] - boxes[:, 1]
        aspect_ratio = w / (h + 1e-6)
        
        valid_mask = (aspect_ratio < 3.0) & (aspect_ratio > 0.3) & (confs > 0.1)
        
        if np.sum(valid_mask) == 0:
            return sv.Detections.empty()
        
        return sv.Detections(
            xyxy=boxes[valid_mask],
            confidence=confs[valid_mask],
            class_id=detections.class_id[valid_mask]
        )
    
    def _novel_postprocessing(self, detections):
        """Enhanced track management with FIXED KalmanFilter"""
        if len(detections) == 0 or detections.tracker_id is None:
            return detections
        
        valid_tracks = detections.tracker_id != -1
        if not np.any(valid_tracks):
            return detections
        
        # Update existing tracks with Kalman + scoring
        valid_tracker_ids = detections.tracker_id[valid_tracks]
        valid_boxes = detections.xyxy[valid_tracks]
        valid_confs = detections.confidence[valid_tracks]
        
        # Update confidences with speed scoring
        enhanced_confs = valid_confs.copy()
        for i, track_id in enumerate(valid_tracker_ids):
            track_id = int(track_id)
            center_x = (valid_boxes[i][0] + valid_boxes[i][2]) / 2
            center_y = (valid_boxes[i][1] + valid_boxes[i][3]) / 2
            
            # Update Kalman (no storage needed for simple enhancement)
            kf = self._get_kalman_filter(track_id, center_x, center_y)
            kf.predict()
            kf.correct(np.array([[center_x], [center_y]], dtype=np.float32))
            
            # Update history and compute speed score
            self._update_track_history(track_id, center_x, center_y, valid_confs[i])
            speed_score = self._compute_speed_score(track_id)
            enhanced_confs[i] *= speed_score
        
        # Update original detections with enhanced confidences
        detections.confidence[valid_tracks] = enhanced_confs
        
        return detections
    
    def _get_kalman_filter(self, track_id, x, y):
        """FIXED: Proper OpenCV KalmanFilter initialization"""
        if track_id not in self.kalman_filters:
            kf = cv2.KalmanFilter(4, 2)
            
            # FIXED: Correct OpenCV property names
            kf.transitionMatrix = np.array([
                [1, 0, 1, 0],
                [0, 1, 0, 1],
                [0, 0, 1, 0],
                [0, 0, 0, 1]
            ], np.float32)
            
            kf.measurementMatrix = np.array([
                [1, 0, 0, 0],
                [0, 1, 0, 0]
            ], np.float32)
            
            kf.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
            kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 1e-1
            
            kf.statePre = np.array([[x], [y], [0], [0]], dtype=np.float32)
            kf.errorCovPre = np.eye(4, dtype=np.float32)
            
            self.kalman_filters[track_id] = kf
        
        return self.kalman_filters[track_id]
    
    def _update_track_history(self, track_id, x, y, conf):
        """Update track trajectory history"""
        if track_id not in self.track_histories:
            self.track_histories[track_id] = deque(maxlen=10)
        self.track_histories[track_id].append((x, y, conf))
    
    def _compute_speed_score(self, track_id):
        """Speed-based confidence scoring"""
        if track_id not in self.track_histories or len(self.track_histories[track_id]) < 2:
            return 1.0
        
        positions = list(self.track_histories[track_id])
        curr_pos = positions[-1][:2]
        prev_pos = positions[-2][:2]
        
        dx = curr_pos[0] - prev_pos[0]
        dy = curr_pos[1] - prev_pos[1]
        speed = np.sqrt(dx**2 + dy**2)
        
        max_drone_speed = 15  # pixels/frame
        speed_score = np.clip(1.0 - (speed / max_drone_speed) * self.speed_factor, 0.3, 1.0)
        return speed_score

# SIMPLIFIED Usage - Works perfectly now!
def run_novel_tracking_inference(model_path, videos_folder, output_folder):
    """Clean inference with FIXED Novel ByteTrack"""
    print("🚀 Loading model...")
    model = YOLO(model_path)
    
    novel_tracker = NovelByteTrack(track_thresh=0.4, track_buffer=30, match_thresh=0.8)
    
    os.makedirs(output_folder, exist_ok=True)
    video_files = list(Path(videos_folder).glob("*.mp4"))
    print(f"📹 Found {len(video_files)} videos")
    
    for video_path in video_files:
        print(f"\n🎬 Novel Tracking: {video_path.name}")
        
        cap = cv2.VideoCapture(str(video_path))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        output_path = os.path.join(output_folder, f"novel_{video_path.stem}.mp4")
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Detection + Novel Tracking
            results = model(frame, conf=0.25, verbose=False)[0]
            detections = sv.Detections.from_ultralytics(results)
            tracked_detections = novel_tracker.update_with_detections(detections)
            
            # Simple visualization
            annotated_frame = frame.copy()
            
            if len(tracked_detections) > 0:
                box_annotator = sv.BoxAnnotator(thickness=2)
                annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=tracked_detections)
                
                labels = [f"#{int(tid)}" if tid != -1 else "" 
                         for tid in tracked_detections.tracker_id]
                label_annotator = sv.LabelAnnotator()
                annotated_frame = label_annotator.annotate(scene=annotated_frame, 
                                                         detections=tracked_detections, 
                                                         labels=labels)
            
            cv2.putText(annotated_frame, f'NovelTrack Frame: {frame_count}', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            
            out.write(annotated_frame)
            frame_count += 1
        
        cap.release()
        out.release()
        print(f"  ✅ {frame_count} frames processed")
        print(f"  💾 Saved: novel_{video_path.stem}.mp4")

# PERFECTLY WORKING!
if __name__ == "__main__":
    MODEL_PATH = "/kaggle/input/enhanced-yolov8-l-on-seadronessee/pytorch/v1/6/last.pt"
    VIDEOS_FOLDER = "/kaggle/working/videos/kaggle/working/videos"
    OUTPUT_FOLDER = "/kaggle/working/novel_tracked_videos"
    
    run_novel_tracking_inference(MODEL_PATH, VIDEOS_FOLDER, OUTPUT_FOLDER)
    print("\n🎉 Novel ByteTrack COMPLETELY FIXED!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Loading model...
📹 Found 12 videos

🎬 Novel Tracking: video_012.mp4
  ✅ 715 frames processed
  💾 Saved: novel_video_012.mp4

🎬 Novel Tracking: video_021.mp4
  ✅ 7870 frames processed
  💾 Saved: novel_video_021.mp4

🎬 Novel Tracking: video_008.mp4
  ✅ 1001 frames processed
  💾 Saved: novel_video_008.mp4

🎬 Novel Tracking: video_007.mp4
  ✅ 1001 frames processed
  💾 Saved: novel_video_007.mp4

🎬 Novel Tracking: video_018.mp4
  ✅ 6028 frames processed
  💾 Saved: novel_video_018.mp4

🎬 Novel Tracking: video_019.mp4
  ✅ 913 frames processed
  💾 Saved: novel_video_019.mp4

🎬 Novel Tracking: video_000.mp4
  ✅ 410 frames processed
  💾 Saved: novel_video_000.mp4

🎬 Novel Tracking: video_